In [41]:
import keras
import numpy as np

In [42]:
path = keras.utils.get_file('nietzche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

606208/600901 [==============================] - 6s 10us/step


In [43]:
text = open(path).read().lower()

In [44]:
print('Corpus Length: ', len(text))

Corpus Length:  600893


In [45]:
maxlen = 60
step = 3
sentences = []
next_chars = []

In [46]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+ maxlen])
    next_chars.append(text[i+maxlen])

In [47]:
print('Number of sequences: ', len(sentences))

Number of sequences:  200278


In [48]:
chars = sorted(list(set(text)))

In [49]:
print('Unique Characters: ', len(chars))

Unique Characters:  57


In [50]:
char_indices = dict((char, char.index(char)) for char in chars)
print('Vectorization ...')

Vectorization ...


In [51]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [52]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [53]:
from keras import layers

In [55]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer = optimizer)          

In [56]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [57]:
import random
import sys
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs = 1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index+maxlen]
    print('Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('-----------temperature: ', temperature)
        sys.stdout.write(generated_text)
        
        
        for i in range(400):
            sampledm = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text+= next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
 87424/200278 [============>.................] - ETA: 1:28 - loss: 0.0059

KeyboardInterrupt: 